# 🚀 Transformers: A Revolução que Mudou a IA Para Sempre!

## Módulo 3: Desvendando a Arquitetura por Trás dos LLMs

**Pedro Nunes Guth - Expert em IA & AWS**

---

Eaí pessoal! 🎯

Lembram do módulo passado quando falamos sobre LLMs? Pois é, agora chegou a hora de entender **COMO** essas máquinas incríveis funcionam por dentro!

É como se no módulo anterior você conheceu um carro incrível (os LLMs), e agora vamos abrir o capô para ver o motor V8 turbinado que faz tudo funcionar: **a Arquitetura Transformer**!

Bora que hoje vai ser **LIIINDO**! 🔥

In [ ]:
# Setup inicial - Preparando nosso laboratório!
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import MultiheadAttention
import math
import warnings
warnings.filterwarnings('ignore')

# Configurações para gráficos mais bonitos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🚀 Ambiente configurado! Prontos para explorar os Transformers!")
print(f"📱 PyTorch versão: {torch.__version__}")
print("🎯 Bora começar a diversão!")

## 🎭 O Que São Transformers Afinal?

Tá, mas o que diabos é um Transformer?

Imagina que você tá numa festa e precisa prestar atenção em várias conversas ao mesmo tempo. Seu cérebro faz isso naturalmente - você consegue:
- Focar na conversa principal
- Captar informações importantes de outras conversas 
- Entender o contexto geral do ambiente
- Conectar informações de diferentes momentos

**É exatamente isso que os Transformers fazem!** 🧠

### Por que Revolutionaram a IA?

Antes dos Transformers (2017), os modelos de linguagem eram como aquele amigo que só consegue prestar atenção numa coisa de cada vez. Eles processavam texto **sequencialmente** - palavra por palavra, da esquerda para direita.

Os Transformers chegaram e disseram: "Ô gente, por que não olhamos para TODAS as palavras ao mesmo tempo e deixamos elas conversarem entre si?"

**BOOM!** 💥 Nasceram o GPT, BERT, ChatGPT e toda essa revolução que vivemos hoje!

### 💡 Dica do Pedro
O nome "Transformer" vem do paper "Attention Is All You Need" (2017). É como se os pesquisadores dissessem: "Esqueçam tudo! A atenção é tudo que precisamos!" E cara... eles estavam certos!

## 🏗️ Visão Geral da Arquitetura

Vamos visualizar como um Transformer funciona de forma geral:

```mermaid
graph TD
    A["📝 Texto de Entrada"] --> B["🔢 Tokenização"]
    B --> C["📊 Embeddings"]
    C --> D["📍 Positional Encoding"]
    D --> E["🎯 Multi-Head Attention"]
    E --> F["🧮 Feed Forward"]
    F --> G["🔄 Camadas Transformer"]
    G --> H["📤 Saída Final"]
    
    style A fill:#ff9999
    style E fill:#99ccff
    style H fill:#99ff99
```

Pensa no Transformer como uma **fábrica super inteligente**:

1. **Entrada**: Texto cru ("Hoje está um belo dia")
2. **Tokenização**: Quebra em pedaços menores
3. **Embeddings**: Transforma palavras em números
4. **Atenção**: As palavras "conversam" entre si
5. **Processamento**: Várias camadas refinam o entendimento
6. **Saída**: Resultado final (próxima palavra, tradução, etc.)

Liiindo né? Agora vamos mergulhar em cada parte! 🏊‍♂️

In [ ]:
# Vamos criar uma visualização da arquitetura Transformer
fig, ax = plt.subplots(figsize=(12, 8))

# Definindo as camadas do Transformer
layers = [
    'Texto de Entrada',
    'Token Embeddings',
    'Positional Encoding',
    'Multi-Head Attention',
    'Add & Norm',
    'Feed Forward',
    'Add & Norm',
    'Saída'
]

colors = ['#ff9999', '#ffcc99', '#99ff99', '#99ccff', 
          '#cc99ff', '#ffff99', '#ff99cc', '#99ffcc']

y_positions = range(len(layers))

# Criando as barras horizontais
bars = ax.barh(y_positions, [1]*len(layers), color=colors, alpha=0.7, height=0.6)

# Adicionando os nomes das camadas
for i, layer in enumerate(layers):
    ax.text(0.5, i, layer, ha='center', va='center', fontweight='bold', fontsize=11)

ax.set_ylim(-0.5, len(layers) - 0.5)
ax.set_xlim(0, 1)
ax.set_yticks([])
ax.set_xticks([])
ax.set_title('🏗️ Arquitetura Transformer - Fluxo de Dados', fontsize=16, fontweight='bold', pad=20)

# Adicionando setas
for i in range(len(layers)-1):
    ax.annotate('', xy=(0.5, i+1), xytext=(0.5, i),
                arrowprops=dict(arrowstyle='->', lw=2, color='black'))

plt.tight_layout()
plt.show()

print("🎯 Essa é a jornada que cada palavra faz dentro de um Transformer!")
print("📚 Nos próximos módulos veremos Tokens e Embeddings em detalhes!")

## 🎯 O Coração dos Transformers: Attention Mechanism

Chegamos na parte mais **IMPORTANTE** de todo o Transformer: o **Mecanismo de Atenção**!

### A Analogia da Festa (Versão Melhorada) 🎉

Imagina que você tá numa festa e alguém grita: "O Pedro chegou!"

Seu cérebro automaticamente:
1. **Processa a frase completa** (não palavra por palavra)
2. **Identifica que "Pedro" é o foco** principal
3. **Conecta "chegou" com "Pedro"** (não com "O")
4. **Entende o contexto** da situação

É isso que a **Atenção** faz! Ela permite que cada palavra "olhe" para todas as outras e decida o quanto deve prestar atenção em cada uma.

### Como Funciona Matematicamente?

A fórmula da atenção é surpreendentemente elegante:

**Attention(Q, K, V) = softmax(QK^T / √d_k)V**

Onde:
- **Q (Query)**: "O que estou procurando?"
- **K (Key)**: "O que tenho disponível?"
- **V (Value)**: "Qual informação vou usar?"

### 💡 Dica do Pedro
Pensa assim: Q é sua pergunta, K são as opções de resposta, e V é o conteúdo real das respostas. A atenção calcula o quanto cada resposta é relevante para sua pergunta!

In [ ]:
# Implementando um mecanismo de atenção simples
def simple_attention(query, key, value, mask=None):
    """
    Implementação simples do mecanismo de atenção
    
    Args:
        query: Tensor [batch_size, seq_len, d_model]
        key: Tensor [batch_size, seq_len, d_model] 
        value: Tensor [batch_size, seq_len, d_model]
        mask: Tensor opcional para mascarar posições
    
    Returns:
        output: Tensor com atenção aplicada
        attention_weights: Pesos de atenção
    """
    d_k = query.size(-1)
    
    # Passo 1: Calcular scores de atenção (Q * K^T)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    
    # Passo 2: Aplicar máscara se fornecida
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    
    # Passo 3: Aplicar softmax para obter probabilidades
    attention_weights = F.softmax(scores, dim=-1)
    
    # Passo 4: Aplicar atenção aos valores
    output = torch.matmul(attention_weights, value)
    
    return output, attention_weights

# Teste com dados sintéticos
batch_size, seq_len, d_model = 1, 4, 8

# Criando tensores de exemplo
Q = torch.randn(batch_size, seq_len, d_model)
K = torch.randn(batch_size, seq_len, d_model)
V = torch.randn(batch_size, seq_len, d_model)

print("🧮 Testando nossa implementação de atenção:")
print(f"📐 Dimensões: batch={batch_size}, seq_len={seq_len}, d_model={d_model}")

output, attention_weights = simple_attention(Q, K, V)

print(f"✅ Saída da atenção: {output.shape}")
print(f"🎯 Pesos de atenção: {attention_weights.shape}")
print("\n🔥 Implementação funcionando perfeitamente!")

In [ ]:
# Vamos visualizar os pesos de atenção!
# Simulando uma frase: "O gato subiu no telhado"
words = ["O", "gato", "subiu", "no", "telhado"]
seq_len = len(words)

# Criando uma matriz de atenção realística
# Onde palavras importantes prestam mais atenção umas nas outras
attention_matrix = np.array([
    [0.1, 0.8, 0.05, 0.03, 0.02],  # "O" presta atenção principalmente em "gato"
    [0.15, 0.4, 0.3, 0.1, 0.05],   # "gato" se conecta com "subiu"
    [0.05, 0.3, 0.4, 0.15, 0.1],   # "subiu" conecta "gato" e "telhado"
    [0.02, 0.1, 0.15, 0.3, 0.43],  # "no" presta atenção em "telhado"
    [0.03, 0.2, 0.2, 0.27, 0.3]    # "telhado" se conecta com várias palavras
])

# Criando o heatmap
fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(attention_matrix, 
            xticklabels=words, 
            yticklabels=words,
            annot=True, 
            fmt='.2f', 
            cmap='Blues',
            cbar_kws={'label': 'Peso de Atenção'},
            ax=ax)

ax.set_title('🎯 Matriz de Atenção: "O gato subiu no telhado"', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Palavras (Keys)', fontsize=12, fontweight='bold')
ax.set_ylabel('Palavras (Queries)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("🔍 Analisando a matriz de atenção:")
print("📈 Valores mais altos = mais atenção entre as palavras")
print("🎯 Note como 'gato' e 'subiu' prestam atenção um no outro!")
print("🏠 'Telhado' recebe atenção de várias palavras (é o destino da ação!)")

## 🧠 Multi-Head Attention: Múltiplas Perspectivas

Tá, mas aqui vem a parte **GENIAL** dos Transformers!

Imagina que você tem **8 cérebros diferentes** analisando a mesma frase, cada um prestando atenção em aspectos diferentes:

- **Cérebro 1**: Foca em relações sujeito-verbo
- **Cérebro 2**: Analisa objetos e complementos  
- **Cérebro 3**: Identifica emoções e sentimentos
- **Cérebro 4**: Procura por negações
- E assim por diante...

No final, você **combina** todas essas perspectivas para ter uma compreensão **MUITO** mais rica do texto!

### Como Funciona?

```mermaid
graph TD
    A["🧠 Entrada"] --> B1["Head 1"]
    A --> B2["Head 2"]
    A --> B3["Head 3"]
    A --> B4["Head ..."]
    A --> B5["Head 8"]
    
    B1 --> C["🔗 Concatenar"]
    B2 --> C
    B3 --> C
    B4 --> C
    B5 --> C
    
    C --> D["📊 Projeção Linear"]
    D --> E["✨ Saída Final"]
    
    style A fill:#ff9999
    style E fill:#99ff99
```

### 💡 Dica do Pedro
É como ter uma mesa redonda com 8 especialistas diferentes analisando o mesmo problema. Cada um traz uma perspectiva única, e a decisão final considera todas as opiniões!

In [ ]:
# Implementando Multi-Head Attention
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        # Projeções lineares para Q, K, V
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        """Implementa a atenção scaled dot-product"""
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
            
        attention_weights = F.softmax(scores, dim=-1)
        output = torch.matmul(attention_weights, V)
        
        return output, attention_weights
    
    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        
        # 1. Aplicar projeções lineares
        Q = self.W_q(query)
        K = self.W_k(key) 
        V = self.W_v(value)
        
        # 2. Dividir em múltiplas heads
        Q = Q.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = K.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = V.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        # 3. Aplicar atenção
        attention_output, attention_weights = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # 4. Concatenar heads
        attention_output = attention_output.transpose(1, 2).contiguous().view(
            batch_size, -1, self.d_model)
        
        # 5. Projeção final
        output = self.W_o(attention_output)
        
        return output, attention_weights

# Testando nossa implementação
d_model = 512
num_heads = 8
seq_len = 10
batch_size = 2

mha = MultiHeadAttention(d_model, num_heads)

# Dados de teste
x = torch.randn(batch_size, seq_len, d_model)

print("🧠 Testando Multi-Head Attention:")
print(f"📊 Entrada: {x.shape}")
print(f"🎯 Número de heads: {num_heads}")
print(f"📐 Dimensão por head: {d_model // num_heads}")

output, attention_weights = mha(x, x, x)

print(f"✅ Saída: {output.shape}")
print(f"⚖️ Pesos de atenção: {attention_weights.shape}")
print("\n🔥 Multi-Head Attention implementado com sucesso!")

In [ ]:
# Visualizando como diferentes heads focam em aspectos diferentes
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

# Simulando padrões de atenção de 8 heads diferentes
words = ["Eu", "amo", "programar", "IA", "com", "Python"]
seq_len = len(words)

# Cada head tem um padrão de atenção diferente
attention_patterns = [
    # Head 1: Foca em pronomes e verbos
    np.array([[0.7, 0.2, 0.05, 0.02, 0.02, 0.01],
              [0.3, 0.4, 0.2, 0.05, 0.03, 0.02],
              [0.1, 0.3, 0.4, 0.1, 0.05, 0.05],
              [0.05, 0.1, 0.3, 0.4, 0.1, 0.05],
              [0.02, 0.05, 0.2, 0.2, 0.4, 0.13],
              [0.01, 0.02, 0.1, 0.15, 0.2, 0.52]]),
    
    # Head 2: Foca em objetos e complementos
    np.array([[0.1, 0.1, 0.3, 0.4, 0.05, 0.05],
              [0.05, 0.2, 0.25, 0.35, 0.1, 0.05],
              [0.02, 0.08, 0.4, 0.3, 0.15, 0.05],
              [0.03, 0.07, 0.3, 0.4, 0.15, 0.05],
              [0.02, 0.03, 0.2, 0.25, 0.3, 0.2],
              [0.01, 0.02, 0.15, 0.2, 0.22, 0.4]]),
    
    # Head 3: Padrão local (palavras adjacentes)
    np.array([[0.5, 0.4, 0.05, 0.02, 0.02, 0.01],
              [0.4, 0.3, 0.25, 0.03, 0.01, 0.01],
              [0.05, 0.25, 0.4, 0.25, 0.03, 0.02],
              [0.02, 0.03, 0.25, 0.4, 0.25, 0.05],
              [0.01, 0.01, 0.03, 0.25, 0.4, 0.3],
              [0.01, 0.01, 0.02, 0.05, 0.31, 0.6]]),
    
    # Mais 5 heads com padrões variados...
    np.random.dirichlet(np.ones(seq_len), seq_len),
    np.random.dirichlet(np.ones(seq_len), seq_len),
    np.random.dirichlet(np.ones(seq_len), seq_len),
    np.random.dirichlet(np.ones(seq_len), seq_len),
    np.random.dirichlet(np.ones(seq_len), seq_len)
]

head_names = [
    "Head 1: Sujeito-Verbo",
    "Head 2: Objetos", 
    "Head 3: Adjacência",
    "Head 4: Padrão A",
    "Head 5: Padrão B",
    "Head 6: Padrão C", 
    "Head 7: Padrão D",
    "Head 8: Padrão E"
]

for i in range(8):
    sns.heatmap(attention_patterns[i],
                xticklabels=words,
                yticklabels=words,
                cmap='Blues',
                ax=axes[i],
                cbar=False,
                square=True)
    axes[i].set_title(head_names[i], fontsize=10, fontweight='bold')
    axes[i].tick_params(labelsize=8)

plt.suptitle('🧠 Multi-Head Attention: 8 Perspectivas Diferentes da Mesma Frase', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("🎯 Cada head aprende a focar em aspectos diferentes!")
print("🔍 Head 1: Conecta sujeitos com verbos")
print("🎪 Head 2: Identifica objetos e complementos") 
print("🔗 Head 3: Analisa palavras próximas")
print("✨ Os outros heads descobrem padrões únicos durante o treinamento!")

## ⚡ Feed Forward Network: O Processamento Individual

Depois que cada palavra "conversou" com todas as outras através da atenção, chegou a hora do **processamento individual**!

É como se após a reunião em grupo, cada pessoa fosse para sua sala pensar individualmente sobre o que discutiu.

### O que faz a Feed Forward Network?

```
FFN(x) = max(0, xW₁ + b₁)W₂ + b₂
```

Traduzindo:
1. **Primeira camada**: Expande a representação (geralmente 4x maior)
2. **ReLU**: Adiciona não-linearidade 
3. **Segunda camada**: Comprime de volta ao tamanho original

### Analogia do Processamento de Texto 📝

Imagina que você recebeu várias informações sobre uma pessoa:
- Nome: Pedro
- Profissão: Instrutor
- Expertise: IA e AWS
- Local: Brasil

A Feed Forward pega essas informações e:
1. **Expande**: Gera várias hipóteses e conexões
2. **Processa**: Analisa padrões e relações
3. **Condensa**: Retorna uma representação refinada

### 💡 Dica do Pedro
A FFN é onde o "pensamento profundo" acontece! Enquanto a atenção conecta informações, a FFN processa e refina cada representação individualmente.

In [ ]:
# Implementando a Feed Forward Network
class FeedForwardNetwork(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(FeedForwardNetwork, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Primeira transformação linear + ReLU
        x = self.relu(self.linear1(x))
        x = self.dropout(x)
        
        # Segunda transformação linear
        x = self.linear2(x)
        
        return x

# Testando a FFN
d_model = 512
d_ff = 2048  # Geralmente 4x maior que d_model
batch_size = 1
seq_len = 8

ffn = FeedForwardNetwork(d_model, d_ff)

# Dados de teste
x = torch.randn(batch_size, seq_len, d_model)

print("⚡ Testando Feed Forward Network:")
print(f"📊 Entrada: {x.shape} (batch_size, seq_len, d_model)")
print(f"🔍 d_model: {d_model}")
print(f"📈 d_ff (dimensão interna): {d_ff}")

output = ffn(x)

print(f"✅ Saída: {output.shape}")
print(f"🎯 Mesma dimensão da entrada: {output.shape == x.shape}")

# Analisando os parâmetros
total_params = sum(p.numel() for p in ffn.parameters())
print(f"\n📊 Parâmetros da FFN: {total_params:,}")
print("🔥 FFN funcionando perfeitamente!")

In [ ]:
# Visualizando o que acontece dentro da FFN
# Vamos analisar como as dimensões mudam ao longo da rede

with torch.no_grad():
    # Entrada
    x = torch.randn(1, 8, 512)
    
    # Primeira camada linear
    x1 = ffn.linear1(x)  # [1, 8, 512] -> [1, 8, 2048]
    
    # Após ReLU
    x2 = ffn.relu(x1)    # Aplica ReLU (zera valores negativos)
    
    # Segunda camada linear
    x3 = ffn.linear2(x2) # [1, 8, 2048] -> [1, 8, 512]

# Visualizando o fluxo de dimensões
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Heatmap da entrada
im1 = axes[0,0].imshow(x[0].numpy(), cmap='RdBu', aspect='auto')
axes[0,0].set_title('📥 Entrada (8 x 512)', fontweight='bold')
axes[0,0].set_ylabel('Sequência')
axes[0,0].set_xlabel('Dimensões')
plt.colorbar(im1, ax=axes[0,0])

# Heatmap após primeira linear (só primeiras 512 dims para visualizar)
im2 = axes[0,1].imshow(x1[0, :, :512].numpy(), cmap='RdBu', aspect='auto')
axes[0,1].set_title('🔄 Após 1ª Linear (8 x 2048→512)', fontweight='bold')
axes[0,1].set_ylabel('Sequência')
axes[0,1].set_xlabel('Dimensões (amostra)')
plt.colorbar(im2, ax=axes[0,1])

# Heatmap após ReLU
im3 = axes[1,0].imshow(x2[0, :, :512].numpy(), cmap='RdBu', aspect='auto')
axes[1,0].set_title('⚡ Após ReLU (8 x 2048→512)', fontweight='bold')
axes[1,0].set_ylabel('Sequência')
axes[1,0].set_xlabel('Dimensões (amostra)')
plt.colorbar(im3, ax=axes[1,0])

# Heatmap da saída
im4 = axes[1,1].imshow(x3[0].numpy(), cmap='RdBu', aspect='auto')
axes[1,1].set_title('📤 Saída Final (8 x 512)', fontweight='bold')
axes[1,1].set_ylabel('Sequência')
axes[1,1].set_xlabel('Dimensões')
plt.colorbar(im4, ax=axes[1,1])

plt.suptitle('⚡ Fluxo de Dados na Feed Forward Network', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("🔍 Análise do fluxo:")
print(f"📥 Entrada: {x.shape}")
print(f"📈 Expansão: {x1.shape} (4x maior!)")
print(f"⚡ Após ReLU: {x2.shape} (zeros valores negativos)")
print(f"📉 Compressão: {x3.shape} (volta ao tamanho original)")
print("\n💡 A FFN expande para processar melhor, depois comprime!")

## 🔗 Residual Connections e Layer Normalization

Agora vamos falar sobre duas técnicas **FUNDAMENTAIS** que fazem os Transformers funcionarem tão bem!

### Residual Connections (Conexões Residuais) 🌉

Imagina que você tá fazendo uma receita complexa, mas quer garantir que não vai perder o sabor original dos ingredientes.

As conexões residuais fazem exatamente isso! Elas **preservam a informação original** enquanto permitem que as camadas adicionem informações novas.

**Fórmula**: `saída = camada(entrada) + entrada`

### Layer Normalization 📏

É como ter um "controlador de qualidade" que garante que os valores não fiquem nem muito grandes nem muito pequenos.

Imagina uma banda onde:
- O **baixista** tá tocando muito baixo
- O **guitarrista** tá muito alto
- A **bateria** tá no volume certo

A Layer Norm é como o **técnico de som** que equilibra tudo para soar harmonioso!

### Por que são importantes?

1. **Residual**: Evita o "vanishing gradient" (gradientes que somem)
2. **Layer Norm**: Estabiliza o treinamento
3. **Juntas**: Permitem redes muito profundas (até 96 camadas no GPT-3!)

### 💡 Dica do Pedro
Sem essas duas técnicas, seria impossível treinar modelos gigantes como GPT-4. Elas são os "superpoderes" que tornaram os LLMs modernos possíveis!

In [ ]:
# Implementando um bloco Transformer completo com Residual + LayerNorm
class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(TransformerBlock, self).__init__()
        
        # Componentes principais
        self.multi_head_attention = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForwardNetwork(d_model, d_ff, dropout)
        
        # Layer Normalizations
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        # Primeiro sub-bloco: Multi-Head Attention + Residual + LayerNorm
        attention_output, _ = self.multi_head_attention(x, x, x, mask)
        attention_output = self.dropout(attention_output)
        x = self.ln1(x + attention_output)  # Residual connection + LayerNorm
        
        # Segundo sub-bloco: Feed Forward + Residual + LayerNorm  
        ff_output = self.feed_forward(x)
        ff_output = self.dropout(ff_output)
        x = self.ln2(x + ff_output)  # Residual connection + LayerNorm
        
        return x

# Testando um bloco Transformer completo
d_model = 512
num_heads = 8
d_ff = 2048
seq_len = 16
batch_size = 2

transformer_block = TransformerBlock(d_model, num_heads, d_ff)

# Dados de teste
x = torch.randn(batch_size, seq_len, d_model)

print("🧱 Testando Bloco Transformer Completo:")
print(f"📊 Entrada: {x.shape}")
print(f"🧠 Heads: {num_heads}")
print(f"⚡ FFN dim: {d_ff}")

# Forward pass
output = transformer_block(x)

print(f"✅ Saída: {output.shape}")
print(f"🎯 Formato preservado: {output.shape == x.shape}")

# Contando parâmetros
total_params = sum(p.numel() for p in transformer_block.parameters())
print(f"\n📊 Parâmetros do bloco: {total_params:,}")
print("🔥 Bloco Transformer completo funcionando!")

In [ ]:
# Vamos visualizar o efeito da Layer Normalization
import torch.nn as nn

# Criando dados com distribuições diferentes (simulando instabilidade)
np.random.seed(42)
torch.manual_seed(42)

# Dados antes da normalização (instáveis)
data_unstable = torch.tensor([
    [100.0, 0.1, 50.0, 0.01],  # Primeira sequência: valores muito variados
    [0.001, 200.0, 0.5, 75.0], # Segunda sequência: também instável
    [1000.0, 0.2, 10.0, 0.1],  # Terceira sequência: extremamente variado
], dtype=torch.float32)

# Aplicando Layer Normalization
layer_norm = nn.LayerNorm(4)  # 4 é o número de features
data_normalized = layer_norm(data_unstable)

# Visualizando a diferença
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Dados antes da normalização
im1 = ax1.imshow(data_unstable.numpy(), cmap='RdYlBu', aspect='auto')
ax1.set_title('📈 Antes da Layer Normalization\n(Valores Instáveis)', fontweight='bold', fontsize=12)
ax1.set_xlabel('Features')
ax1.set_ylabel('Sequências')
ax1.set_xticks(range(4))
ax1.set_yticks(range(3))
plt.colorbar(im1, ax=ax1, label='Valores')

# Adicionando valores nas células
for i in range(3):
    for j in range(4):
        ax1.text(j, i, f'{data_unstable[i,j]:.3f}', 
                ha='center', va='center', fontweight='bold')

# Dados após normalização
im2 = ax2.imshow(data_normalized.detach().numpy(), cmap='RdYlBu', aspect='auto')
ax2.set_title('📏 Após Layer Normalization\n(Valores Estabilizados)', fontweight='bold', fontsize=12)
ax2.set_xlabel('Features')
ax2.set_ylabel('Sequências')
ax2.set_xticks(range(4))
ax2.set_yticks(range(3))
plt.colorbar(im2, ax=ax2, label='Valores')

# Adicionando valores nas células
for i in range(3):
    for j in range(4):
        ax2.text(j, i, f'{data_normalized[i,j]:.3f}', 
                ha='center', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Estatísticas
print("📊 Estatísticas ANTES da Layer Norm:")
print(f"🔢 Média por linha: {data_unstable.mean(dim=1)}")
print(f"📈 Desvio padrão por linha: {data_unstable.std(dim=1)}")
print(f"📏 Range de valores: {data_unstable.min():.3f} a {data_unstable.max():.3f}")

print("\n📊 Estatísticas APÓS a Layer Norm:")
print(f"🔢 Média por linha: {data_normalized.mean(dim=1)}")
print(f"📈 Desvio padrão por linha: {data_normalized.std(dim=1)}")
print(f"📏 Range de valores: {data_normalized.min():.3f} a {data_normalized.max():.3f}")

print("\n💡 Viu como a Layer Norm estabilizou os valores?")
print("🎯 Média ≈ 0 e desvio padrão ≈ 1 para cada linha!")

## 📍 Positional Encoding: Ensinando Posição aos Transformers

Aqui temos um **PROBLEMA INTERESSANTE**: os Transformers processam todas as palavras ao mesmo tempo, mas como eles sabem a **ordem** das palavras?

Pensa na diferença entre:
- "O gato subiu no telhado" 🐱⬆️
- "O telhado subiu no gato" 🏠⬆️😱

A ordem das palavras **IMPORTA MUITO**!

### A Solução: Positional Encoding 🗺️

É como dar um **GPS** para cada palavra! Cada posição recebe um "código único" que é somado ao embedding da palavra.

### Como Funciona?

```
PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
```

Parece complicado? Na verdade é **GENIAL**! 🤯

### Analogia da Impressão Digital 👆

Imagina que cada posição na frase tem uma "impressão digital" única feita de ondas seno e cosseno:
- **Posição 0**: Um padrão único de ondas
- **Posição 1**: Outro padrão levemente diferente  
- **Posição 2**: Mais um padrão único
- E assim por diante...

### 💡 Dica do Pedro
O uso de seno e cosseno é brilhante! Permite ao modelo aprender facilmente distâncias relativas entre posições. É matemática pura a serviço da IA!

In [ ]:
# Implementando Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length=5000):
        super(PositionalEncoding, self).__init__()
        
        # Criando matriz de positional encodings
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        
        # Calculando os divisores para as frequências
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
                           (-math.log(10000.0) / d_model))
        
        # Aplicando seno nas posições pares e cosseno nas ímpares
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Adicionando dimensão de batch
        pe = pe.unsqueeze(0).transpose(0, 1)
        
        # Registrando como buffer (não é parâmetro treinável)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        # x shape: [batch_size, seq_len, d_model]
        seq_len = x.size(1)
        return x + self.pe[:seq_len, :].transpose(0, 1)

# Testando Positional Encoding
d_model = 512
max_seq_length = 100
batch_size = 1
seq_len = 20

pos_encoding = PositionalEncoding(d_model, max_seq_length)

# Dados de teste (simulando embeddings)
embeddings = torch.randn(batch_size, seq_len, d_model)

print("📍 Testando Positional Encoding:")
print(f"📊 Embeddings originais: {embeddings.shape}")
print(f"🗺️ Comprimento máximo suportado: {max_seq_length}")

# Aplicando positional encoding
embeddings_with_pos = pos_encoding(embeddings)

print(f"✅ Embeddings + Posição: {embeddings_with_pos.shape}")
print(f"🎯 Formato preservado: {embeddings_with_pos.shape == embeddings.shape}")

# Verificando que o positional encoding foi adicionado
difference = embeddings_with_pos - embeddings
print(f"\n📏 Diferença (positional encoding): min={difference.min():.4f}, max={difference.max():.4f}")
print("🔥 Positional Encoding implementado com sucesso!")

In [ ]:
# Visualizando os padrões de Positional Encoding
d_model = 128  # Usando dimensão menor para visualizar melhor
seq_len = 50

# Criando positional encoding
pe_viz = PositionalEncoding(d_model, seq_len)

# Extraindo apenas o positional encoding (sem somar aos embeddings)
pos_encodings = pe_viz.pe[:seq_len, 0, :].numpy()  # [seq_len, d_model]

# Criando visualizações
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Heatmap completo do positional encoding
im1 = ax1.imshow(pos_encodings.T, cmap='RdBu', aspect='auto')
ax1.set_title('🗺️ Positional Encodings Completos\n(Dimensões vs Posições)', fontweight='bold')
ax1.set_xlabel('Posição na Sequência')
ax1.set_ylabel('Dimensões do Modelo')
plt.colorbar(im1, ax=ax1)

# 2. Padrões de algumas dimensões específicas
positions = np.arange(seq_len)
dimensions_to_show = [0, 1, 10, 11, 30, 31]
colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']

for i, dim in enumerate(dimensions_to_show):
    ax2.plot(positions, pos_encodings[:, dim], 
             label=f'Dim {dim}', color=colors[i], linewidth=2)

ax2.set_title('📊 Padrões de Ondas por Dimensão', fontweight='bold')
ax2.set_xlabel('Posição')
ax2.set_ylabel('Valor do Encoding')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Comparando frequências baixas vs altas
ax3.plot(positions, pos_encodings[:, 0], 'r-', linewidth=3, label='Dim 0 (freq. baixa)')
ax3.plot(positions, pos_encodings[:, 1], 'r--', linewidth=3, label='Dim 1 (freq. baixa)')
ax3.plot(positions, pos_encodings[:, 60], 'b-', linewidth=3, label='Dim 60 (freq. alta)')
ax3.plot(positions, pos_encodings[:, 61], 'b--', linewidth=3, label='Dim 61 (freq. alta)')

ax3.set_title('🌊 Frequências Baixas vs Altas', fontweight='bold')
ax3.set_xlabel('Posição')
ax3.set_ylabel('Valor do Encoding')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Similaridade entre posições (produto escalar)
# Calculando similaridade entre posição 10 e todas as outras
reference_pos = 10
similarities = []

for pos in range(seq_len):
    similarity = np.dot(pos_encodings[reference_pos], pos_encodings[pos])
    similarities.append(similarity)

ax4.plot(positions, similarities, 'g-', linewidth=3)
ax4.axvline(x=reference_pos, color='red', linestyle='--', linewidth=2, 
           label=f'Posição de referência ({reference_pos})')
ax4.set_title(f'🎯 Similaridade com Posição {reference_pos}', fontweight='bold')
ax4.set_xlabel('Posição')
ax4.set_ylabel('Similaridade (produto escalar)')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🔍 Análise dos Positional Encodings:")
print("🌊 Frequências baixas: Mudam lentamente, capturam posições distantes")
print("⚡ Frequências altas: Mudam rapidamente, distinguem posições próximas")
print("🎯 Cada posição tem uma 'assinatura' única!")
print("📏 A similaridade mostra como posições próximas são mais similares")

## 🏗️ Construindo um Transformer Completo

Agora chegou a hora do **GRAND FINALE**! Vamos juntar todas as peças e construir um Transformer completo! 🎯

É como montar um quebra-cabeça gigante onde cada peça que estudamos se encaixa perfeitamente:

```mermaid
graph TD
    A["📝 Input Tokens"] --> B["📊 Token Embeddings"]
    B --> C["📍 + Positional Encoding"]
    C --> D["🧱 Transformer Block 1"]
    D --> E["🧱 Transformer Block 2"]
    E --> F["🧱 Transformer Block N"]
    F --> G["📤 Output Layer"]
    
    subgraph "🧱 Transformer Block"
        H["🎯 Multi-Head Attention"]
        I["➕ Add & Norm"]
        J["⚡ Feed Forward"]
        K["➕ Add & Norm"]
        H --> I --> J --> K
    end
    
    style A fill:#ff9999
    style G fill:#99ff99
```

### Arquitetura do Nosso Transformer 🏗️

Vamos criar um **GPT simplificado** (decoder-only) que pode:
- Receber uma sequência de tokens
- Processar através de múltiplas camadas
- Gerar probabilidades para o próximo token

### 💡 Dica do Pedro
Esse é o mesmo princípio por trás do ChatGPT! Claro, com algumas diferenças de escala (eles têm 96+ camadas e trilhões de parâmetros), mas a arquitetura base é essa!

In [ ]:
# Construindo um Transformer completo (GPT-style)
class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout=0.1):
        super(SimpleTransformer, self).__init__()
        
        self.d_model = d_model
        self.vocab_size = vocab_size
        
        # Embeddings
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        
        # Transformer Blocks
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        # Final layer normalization
        self.ln_final = nn.LayerNorm(d_model)
        
        # Output projection to vocabulary
        self.output_projection = nn.Linear(d_model, vocab_size)
        
        # Initialize weights
        self.init_weights()
        
    def init_weights(self):
        """Inicialização dos pesos seguindo as melhores práticas"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
                if module.bias is not None:
                    torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def create_causal_mask(self, seq_len):
        """Cria máscara causal para impedir que o modelo veja tokens futuros"""
        mask = torch.tril(torch.ones(seq_len, seq_len))
        return mask.unsqueeze(0).unsqueeze(0)  # [1, 1, seq_len, seq_len]
    
    def forward(self, input_ids, targets=None):
        batch_size, seq_len = input_ids.shape
        
        # 1. Token embeddings
        token_embeddings = self.token_embedding(input_ids)  # [batch, seq_len, d_model]
        
        # 2. Add positional encoding
        x = self.positional_encoding(token_embeddings)
        
        # 3. Create causal mask (para GPT-style autoregressive generation)
        causal_mask = self.create_causal_mask(seq_len)
        
        # 4. Pass through transformer blocks
        for transformer_block in self.transformer_blocks:
            x = transformer_block(x, causal_mask)
        
        # 5. Final layer normalization
        x = self.ln_final(x)
        
        # 6. Project to vocabulary size
        logits = self.output_projection(x)  # [batch, seq_len, vocab_size]
        
        # 7. Calculate loss if targets provided
        loss = None
        if targets is not None:
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                targets.view(-1),
                ignore_index=-1
            )
        
        return logits, loss

# Configurações do modelo
config = {
    'vocab_size': 1000,      # Vocabulário pequeno para teste
    'd_model': 256,          # Dimensão menor para ser mais rápido
    'num_heads': 8,          # 8 cabeças de atenção
    'num_layers': 6,         # 6 camadas (GPT-2 small tem 12)
    'd_ff': 1024,           # FFN dimension (4x d_model)
    'max_seq_length': 128,   # Sequência máxima
    'dropout': 0.1
}

# Criando nosso Transformer!
model = SimpleTransformer(**config)

# Calculando número de parâmetros
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("🚀 TRANSFORMER COMPLETO CRIADO!")
print(f"📊 Parâmetros totais: {total_params:,}")
print(f"🎯 Parâmetros treináveis: {trainable_params:,}")
print(f"🧠 Número de camadas: {config['num_layers']}")
print(f"🎭 Número de heads: {config['num_heads']}")
print(f"📚 Tamanho do vocabulário: {config['vocab_size']}")
print("\n🔥 Pronto para processar texto como um GPT!")

In [ ]:
# Testando nosso Transformer com dados sintéticos
batch_size = 2
seq_len = 20

# Criando sequências de tokens aleatórios
input_ids = torch.randint(0, config['vocab_size'], (batch_size, seq_len))
targets = torch.randint(0, config['vocab_size'], (batch_size, seq_len))

print("🧪 Testando nosso Transformer:")
print(f"📥 Input shape: {input_ids.shape}")
print(f"🎯 Targets shape: {targets.shape}")

# Forward pass
with torch.no_grad():
    logits, loss = model(input_ids, targets)

print(f"\n📤 Logits shape: {logits.shape}")
print(f"💸 Loss: {loss.item():.4f}")

# Calculando probabilidades para o último token
last_token_logits = logits[0, -1, :]  # Primeiro batch, último token
probabilities = F.softmax(last_token_logits, dim=-1)

# Encontrando os top-5 tokens mais prováveis
top5_probs, top5_indices = torch.topk(probabilities, 5)

print("\n🏆 Top 5 próximos tokens mais prováveis:")
for i, (token_id, prob) in enumerate(zip(top5_indices, top5_probs)):
    print(f"{i+1}. Token {token_id.item()}: {prob.item():.4f} ({prob.item()*100:.2f}%)")

print("\n✅ Transformer funcionando perfeitamente!")
print("🎉 Parabéns! Você acabou de criar e testar um modelo Transformer completo!")

## 🎯 Exercício Prático: Construa Seu Próprio Mini-Transformer!

Agora é a **SUA VEZ** de brilhar! 🌟

### Desafio 1: Implementação Básica

Complete a implementação de um **Mini-Transformer** com as seguintes especificações:
- 2 camadas
- 4 heads de atenção  
- Vocabulário de 500 tokens
- Dimensão de 128

### Dicas para o Sucesso:
1. Use os componentes que já implementamos
2. Teste com sequências pequenas primeiro
3. Verifique se as dimensões estão corretas
4. Monitore se o loss diminui (sinal de que está aprendendo)

### 💡 Dica do Pedro
Comece pequeno e vá crescendo! É melhor ter um modelo simples funcionando do que um complexo quebrado. Lembre-se: mesmo o GPT-4 começou com conceitos básicos como esses!

In [ ]:
# EXERCÍCIO 1: Complete a implementação do Mini-Transformer

class MiniTransformer(nn.Module):
    def __init__(self):
        super(MiniTransformer, self).__init__()
        
        # TODO: Defina os parâmetros do modelo
        self.vocab_size = 500
        self.d_model = 128
        self.num_heads = 4
        self.num_layers = 2
        self.d_ff = 512  # 4x d_model
        self.max_seq_length = 64
        
        # TODO: Implementar os componentes
        # Dica: Use as classes que já criamos acima!
        
        # 1. Token embeddings
        self.token_embedding = None  # COMPLETE AQUI
        
        # 2. Positional encoding
        self.pos_encoding = None  # COMPLETE AQUI
        
        # 3. Transformer blocks
        self.blocks = None  # COMPLETE AQUI
        
        # 4. Output layer
        self.ln_final = None  # COMPLETE AQUI
        self.output_proj = None  # COMPLETE AQUI
    
    def forward(self, x):
        # TODO: Implementar o forward pass
        # 1. Token embeddings
        # 2. Positional encoding
        # 3. Passar pelas camadas transformer
        # 4. Layer norm final
        # 5. Projeção para vocabulário
        
        pass  # SUBSTITUA POR SUA IMPLEMENTAÇÃO

# Teste seu modelo aqui!
print("🎯 EXERCÍCIO 1: Implemente o MiniTransformer acima!")
print("💡 Use as classes que já criamos como base")
print("🚀 Quando terminar, descomente o código abaixo para testar:")

# Descomente quando completar a implementação:
# mini_model = MiniTransformer()
# test_input = torch.randint(0, 500, (1, 10))
# output = mini_model(test_input)
# print(f"✅ Saída: {output.shape}")

## 🎯 Exercício Avançado: Análise de Atenção

### Desafio 2: Visualização Inteligente

Agora vamos fazer algo **MUITO LEGAL**: analisar como nosso Transformer presta atenção em diferentes partes de uma frase!

Crie uma função que:
1. Receba uma frase tokenizada
2. Passe pelo modelo
3. Extraia os padrões de atenção
4. Visualize quais palavras prestam atenção em quais outras

### O que você vai descobrir:
- Como o modelo conecta sujeitos com verbos
- Quais heads focam em que tipos de relações
- Padrões interessantes de linguagem

### 💡 Dica do Pedro
Esse tipo de análise é usado pelos pesquisadores do Google, OpenAI e Meta para entender como os modelos "pensam"! Você vai estar fazendo ciência de ponta!

In [ ]:
# EXERCÍCIO 2: Análise de Padrões de Atenção

def analyze_attention_patterns(model, input_ids, layer_idx=0, head_idx=0):
    """
    Extrai e visualiza padrões de atenção de uma camada específica
    
    Args:
        model: Modelo Transformer
        input_ids: Tokens de entrada
        layer_idx: Qual camada analisar
        head_idx: Qual head analisar
    """
    
    # TODO: Implementar extração de atenção
    # Dica: Você precisa modificar o forward pass para retornar 
    # os pesos de atenção além dos logits
    
    model.eval()
    with torch.no_grad():
        # TODO: Forward pass e extração dos pesos de atenção
        pass
    
    # TODO: Criar visualização dos padrões
    # Dica: Use seaborn.heatmap como fizemos antes
    
    pass

def find_interesting_patterns(attention_weights):
    """
    Encontra padrões interessantes nos pesos de atenção
    
    Args:
        attention_weights: Matriz de atenção [seq_len, seq_len]
        
    Returns:
        dict: Padrões encontrados
    """
    
    patterns = {}
    
    # TODO: Implementar detecção de padrões
    # Exemplos de padrões para detectar:
    # 1. Atenção predominantemente local (diagonal)
    # 2. Atenção em posições específicas
    # 3. Padrões de long-range dependencies
    
    return patterns

print("🔍 EXERCÍCIO 2: Análise de Atenção")
print("💭 Implementar as funções acima para analisar como o modelo 'pensa'")
print("🎨 Criar visualizações que mostrem padrões de atenção")
print("🧠 Descobrir insights sobre o comportamento do modelo!")

# Exemplo de uso (quando implementado):
# sentence_tokens = torch.tensor([[1, 45, 123, 67, 89, 2]])  # Exemplo
# analyze_attention_patterns(model, sentence_tokens, layer_idx=0, head_idx=0)

## 🎊 Resumo: A Jornada pelos Transformers

**PARABÉNS!** 🎉 Você acabou de fazer uma jornada **INCRÍVEL** pelo coração dos LLMs!

### 🧠 O que Aprendemos Hoje:

#### 1. **Fundamentos dos Transformers**
- ✅ Como funcionam diferente dos modelos sequenciais
- ✅ Por que revolucionaram a IA
- ✅ Arquitetura geral e fluxo de dados

#### 2. **Mecanismo de Atenção** 🎯
- ✅ Self-attention e como palavras "conversam"
- ✅ Query, Key, Value - a matemática por trás
- ✅ Multi-Head Attention - múltiplas perspectivas

#### 3. **Componentes Essenciais** ⚙️
- ✅ Feed Forward Networks - processamento individual
- ✅ Residual Connections - preservando informação
- ✅ Layer Normalization - estabilizando treinamento
- ✅ Positional Encoding - ensinando posição

#### 4. **Implementação Prática** 💻
- ✅ Código funcional de cada componente
- ✅ Transformer completo do zero
- ✅ Testes e validações

### 🔗 Conectando com o Curso:

**Módulos anteriores:**
- ✅ **Módulo 1**: Setup que usamos hoje ✨
- ✅ **Módulo 2**: LLMs que agora sabemos como funcionam! 🧠

**Próximos módulos:**
- 🔜 **Módulo 4**: Tokens e Tokenização - como texto vira números
- 🔜 **Módulo 5**: Embeddings - representações semânticas
- 🔜 **Módulo 6**: Tipos de Modelos - GPT, BERT, T5 e família

### 💡 Dica Final do Pedro:
Os Transformers são a base de **TUDO** na IA moderna: ChatGPT, Claude, Gemini, todos usam essa arquitetura! Você agora entende o "motor" por trás da revolução da IA. **LIIINDO!** 🚀

### 🎯 Para Casa:
1. Complete os exercícios se não terminou
2. Experimente com diferentes configurações
3. Pense em como isso se conecta com os LLMs que usa no dia a dia

**Nos vemos no próximo módulo para falar sobre Tokenização!** 🔥

In [ ]:
# 🎊 Celebração final - Estatísticas do que construímos!

def model_summary(model):
    """Cria um resumo detalhado do modelo"""
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    # Calculando tamanho aproximado em MB
    param_size_mb = total_params * 4 / (1024 * 1024)  # 4 bytes por parâmetro float32
    
    print("🏆 RESUMO DO SEU TRANSFORMER")
    print("=" * 50)
    print(f"📊 Parâmetros totais: {total_params:,}")
    print(f"🎯 Parâmetros treináveis: {trainable_params:,}")
    print(f"💾 Tamanho aproximado: {param_size_mb:.2f} MB")
    print(f"🧠 Camadas: {len(model.transformer_blocks)}")
    print(f"🎭 Heads por camada: {model.transformer_blocks[0].multi_head_attention.num_heads}")
    print(f"📚 Vocabulário: {model.vocab_size:,} tokens")
    print(f"📏 Dimensão do modelo: {model.d_model}")
    
    # Comparação com modelos famosos
    print("\n🔍 COMPARAÇÃO COM MODELOS FAMOSOS:")
    print("=" * 50)
    
    famous_models = {
        "Seu Transformer": total_params,
        "GPT-2 Small": 124_000_000,
        "GPT-2 Medium": 355_000_000, 
        "GPT-2 Large": 774_000_000,
        "GPT-3": 175_000_000_000,
        "GPT-4": 1_760_000_000_000  # Estimativa
    }
    
    for name, params in famous_models.items():
        if name == "Seu Transformer":
            print(f"🚀 {name}: {params:,} parâmetros ⭐")
        else:
            ratio = params / total_params
            print(f"📈 {name}: {params:,} parâmetros ({ratio:.0f}x maior)")
    
    print("\n🎉 PARABÉNS! Você construiu um Transformer funcional!")
    print("💡 É o mesmo princípio do ChatGPT, só que em escala menor!")
    
model_summary(model)

print("\n" + "="*60)
print("🎊 MÓDULO 3 CONCLUÍDO COM SUCESSO! 🎊")
print("🚀 Você agora é um expert em Arquitetura Transformer!")
print("🔥 Preparado para os próximos desafios do curso!")
print("📚 Próximo stop: Tokens e Tokenização!")
print("="*60)